In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
#adjust max_columns

In [4]:
pd.options.display.max_columns = 50

In [5]:
#load data

In [2618]:
df1 = pd.read_excel('./课题数据/基本信息.xls')

In [7]:
df2 = pd.read_excel('./课题数据/实验室胚胎移植记录结果.xls')

In [28]:
df3 = pd.read_excel('./课题数据/检查检验结果2.xls')

In [2526]:
df4 = pd.read_excel('./课题数据/按身份证关联_产科分娩信息+高危.xlsx')

In [ ]:
#rename columns

In [2619]:
df1 = df1.rename(columns={'病案号':'IVF编号'})

In [29]:
df3 = df3.rename(columns={'ivf编号':'IVF编号'})

### 对【基本信息表】进行处理

In [ ]:
#1.去重与合并

In [2620]:
df1['孕育史'] = df1['孕育史'].apply(str) #填充NA方便分组内字符串的合并

In [2621]:
#按照IVF编号分组，对每个分组合并诊断和孕育史字符串
strConcatPerIVF = df1[['IVF编号', '诊断','孕育史']].groupby('IVF编号')['诊断'].apply(lambda x: ', '.join(x)).reset_index()

In [2622]:
strConcatPerIVF['孕育史'] = df1[['IVF编号', '诊断','孕育史']].groupby('IVF编号')['孕育史']\
                            .apply(lambda x: ', '.join(x)).reset_index(drop=True)
#按照IVF编号分组，对每个分组合并诊断和孕育史字符串

In [2623]:
df1_pro = df1[['IVF编号','女方姓名','女方生日','男方姓名','男方生日']].drop_duplicates().merge(strConcatPerIVF, on='IVF编号', how='left')
#只选出以上五项内容进行去重，合并诊断和孕育史信息

In [2626]:
df1_pro.describe()
#发现得到的表格行数大于IVF编号数目，即有部分重复的IVF未去重
#该人的IVF为17-1756

,IVF编号,女方姓名,女方生日,男方姓名,男方生日,诊断,孕育史
count,8771,8752,8746,8752,8751,8771,8771
unique,8770,7919,4777,8116,5085,143,126
top,17-1756,张婷,1984-01-01 00:00:00,张俊,1978-01-01 00:00:00,",,,",0-0-0-0
freq,2,12,22,9,16,3354,4530
first,NaN,NaN,1963-05-25 00:00:00,NaN,1900-01-01 00:00:00,NaN,NaN
last,NaN,NaN,1997-09-16 00:00:00,NaN,1999-05-13 00:00:00,NaN,NaN


In [2627]:
df1_pro[df1_pro['IVF编号'] == '17-1756'] #确认该IVF号出现的问题，发现是女方姓名错误地输入为男方姓名

,IVF编号,女方姓名,女方生日,男方姓名,男方生日,诊断,孕育史
7257,17-1756,阳敬宇,1994-09-28,蒯圣杰,1992-10-05,",复发性流产，染色体异常，,,, ,复发性流产，染色体异常，,,","nan, nan"
7258,17-1756,蒯圣杰,1992-10-05,蒯圣杰,1992-10-05,",复发性流产，染色体异常，,,, ,复发性流产，染色体异常，,,","nan, nan"


In [2628]:
df1_pro = df1_pro.drop(7258).reset_index(drop=True) #删除该行

In [2629]:
df1_pro['女方姓名'].describe()
#检查发现女方姓名缺了很多，由于产科数据匹配时只能依赖姓名，故需要尽量补全。

count     8751
unique    7918
top         张婷
freq        12
Name: 女方姓名, dtype: object

In [2630]:
#选出df2中的名字列用以填补基本信息表中缺失
df2_name = df2[['IVF编号','姓名']].drop_duplicates(subset=['IVF编号']).reset_index(drop=True)
#同时要保证这些名字字符串中没有两端没有空白
df2_name.loc[df2_name['姓名'].notna(),'姓名'] = df2_name.loc[df2_name['姓名'].notna(),'姓名'].apply(lambda x: x.strip())

In [2631]:
#修补基本信息中的姓名
df1_name = df1_pro[['IVF编号','女方姓名']].merge(df2_name,on='IVF编号',how='left')
#保证原来列中的名字字符串中没有两端没有空白
df1_name.loc[df1_name['女方姓名'].notna(),'女方姓名'] = df1_name.loc[df1_name['女方姓名'].notna(),'女方姓名'].apply(lambda x: x.strip())

In [2632]:
#填补df1中没有，但是df2中有的
df1_name.loc[df1_name['女方姓名'].isna(),'女方姓名'] = df1_name.loc[df1_name['女方姓名'].isna(),'姓名']

In [2633]:
#检查：发现还有一些不同的，手动更改
df1_name[(df1_name['女方姓名']!= df1_name['姓名']) & (df1_name['姓名'].notna())]

,IVF编号,女方姓名,姓名
3246,16-0110,沈燕★01,沈燕
3351,16-0215,金文,金文
3839,16-0704,谭晓洁,谭晓灇
4705,16-1574,曹晓莉★01,曹晓莉
5916,17-0394,朱晓婷,徐佳春
8121,18-0458,刘霞,刘霞★01


In [2634]:
#手动更正
df1_name.iloc[3246,1] = '沈燕'
df1_name.iloc[3351,1] = '金文'
df1_name.iloc[4705,1] = '曹晓莉'

In [2635]:
#将df1_name中的女方姓名填补到df1_pro中去
df1_pro['女方姓名'] = df1_name['女方姓名']

In [2637]:
#得到了最终基本信息结果 IVF编号为键
df1_pro.describe()

,IVF编号,女方姓名,女方生日,男方姓名,男方生日,诊断,孕育史
count,8770,8764,8745,8751,8750,8770,8770
unique,8770,7926,4776,8116,5085,143,126
top,18-0960,陈洁,1984-01-01 00:00:00,张俊,1978-01-01 00:00:00,",,,",0-0-0-0
freq,1,12,22,9,16,3354,4530
first,NaN,NaN,1963-05-25 00:00:00,NaN,1900-01-01 00:00:00,NaN,NaN
last,NaN,NaN,1997-09-16 00:00:00,NaN,1999-05-13 00:00:00,NaN,NaN


### 对【精液检查结果表】进行处理

#### 1.基本处理

In [476]:
df3 = df3.sort_values(['IVF编号','检查时间']) #重新排序

df3 = df3.reset_index(drop=True) #重新设置index

df3['1类实验结果'] = df3['取精方式'].notna() | df3['精子密度'].notna() #有取精方式或者精子密度的数据条目可以归类为第一类实验结果

df3['2类实验结果'] = df3['形态正常'].notna() #有形态正常数据的归为第二类

df3['3类实验结果'] = df3['DNA碎片率指数'].notna() #有DNA碎片率指数的归为第三类

df3['检查时间差'] = df3[['IVF编号','检查时间']].groupby('IVF编号')['检查时间'].diff() #按IVF分组，计算每组内条目间时间差

df3_sft = df3[['IVF编号','1类实验结果','2类实验结果','3类实验结果']].groupby('IVF编号').shift(1)
#按IVF分组，将每组结果布尔值向下位移，方便之后进行行间比较

df3['能否与上行合并'] = ((df3['1类实验结果'] == False) | (df3_sft['1类实验结果'] == False)) \
& ((df3['2类实验结果'] == False) | (df3_sft['2类实验结果'] == False)) \
& ((df3['3类实验结果'] == False) | (df3_sft['3类实验结果'] == False)) 
#进行行间比较，如果该行与上一行之间不是同一类实验结果，则从数据来讲可以合并（不考虑其检查时间差）

df3_sft2 = df3[['IVF编号','1类实验结果','2类实验结果','3类实验结果']].groupby('IVF编号').shift(2)
#位移2位，方便进行隔行间比较

df3['能否与上上行合并'] = ((df3['1类实验结果'] == False) | (df3_sft2['1类实验结果'] == False)) \
& ((df3['2类实验结果'] == False) | (df3_sft2['2类实验结果'] == False)) \
& ((df3['3类实验结果'] == False) | (df3_sft2['3类实验结果'] == False)) 
#进行隔行比较，如果该行与上上行之间不是同一类实验结果，则从数据来讲可以合并（不考虑其检查时间差）

In [443]:
#得到新的df3

#### 2.找到每一条检查记录的实验组开始的日期（找出组头）

#### 第1次分组

In [477]:
#作为组头的条件：与上方数据结构重复 或 检查数据是患者首行数据 或 该检查与上次检查差30天以上
mycrit = (df3['能否与上行合并'] == False) | (df3['检查时间差'].isna()) | (df3['检查时间差'] >= pd.Timedelta(30, unit='d'))

In [478]:
#get_grouphead(df,mycrit)
#筛选出每组的标记行（即可作为组内数据合并的标杆行）
df3_key = df3[mycrit][['IVF编号','检查时间']]
df3_key.columns=['IVF编号','检查组开始时间']

In [479]:
#get_groups(df,df_grouphead)
#将每条实验记录附上可能的标记行
df3_mgd = df3.merge(df3_key, on='IVF编号', how='left')
#添加记录实验时间和标记实验组开始时间的时间差
df3_mgd['时间差'] = df3_mgd['检查时间'] - df3_mgd['检查组开始时间']
#根据条件筛选出与每条实验记录最匹配的标记行（即该组实验开始日期）；按照时间差排序，保留时间差最接近0的正值，即可得到最匹配的标记行
df3_mgd = df3_mgd[df3_mgd['时间差'] >= pd.Timedelta(0,unit='d')]\
        .sort_values(['IVF编号','检查时间','时间差']).drop_duplicates(subset=['IVF编号','检查时间'])
#重新设置index
df3_mgd = df3_mgd.reset_index(drop=True)

In [480]:
#write_groupinfo(df,mycrit)
#将分组结果写入表格
df3_fin = df3_mgd.copy()
df3_fin['第1次分组结果'] = mycrit
df3_fin['上行第1次分组结果'] = df3_fin[['IVF编号','第1次分组结果']].groupby('IVF编号')[['第1次分组结果']].shift(1) #将上行的分组结果下移方便对比
df3_fin['上上行第1次分组结果'] = df3_fin[['IVF编号','第1次分组结果']].groupby('IVF编号')[['第1次分组结果']].shift(2) #将上上行的下移方便对比

#### 第二次分组：
经观察分组结果，发现过分合并十分严重，原因是时间阈值过大，许多数据行隔行重复同样被合并。
在此我们1.找出组内行数大于等于3的组中的第3行，2.判断其与该组第1行的咬合性，若两者冲突，则将该第3行定为新的组头；3.将新组头与第一次结果合并

In [483]:
mycrit2 = (((df3_fin['第1次分组结果'] == False) & (df3_fin['上行第1次分组结果'] == False) & (df3_fin['上上行第1次分组结果'] == True)\
        & (df3_fin['能否与上上行合并'] == False)) | (df3_fin['第1次分组结果'] == True))

In [484]:
#write_groupinfo
#将分组结果写入表格
df3_fin['第2次分组结果'] = mycrit2
df3_fin['上行第2次分组结果'] = df3_fin[['IVF编号','第2次分组结果']].groupby('IVF编号')[['第2次分组结果']].shift(1) #将上行的分组结果下移方便对比
df3_fin['上上行第2次分组结果'] = df3_fin[['IVF编号','第2次分组结果']].groupby('IVF编号')[['第2次分组结果']].shift(2) #将上上行的下移方便对比

In [518]:
mycrit2 = (((df3_fin['第2次分组结果'] == False) & (df3_fin['上行第2次分组结果'] == False) & (df3_fin['上上行第2次分组结果'] == True)\
        & (df3_fin['能否与上上行合并'] == False)) | (df3_fin['第2次分组结果'] == True))

In [519]:
#根据第2次分组结果再进行一次迭代
df3_fin['第2次分组结果'] = mycrit2
df3_fin['上行第2次分组结果'] = df3_fin[['IVF编号','第2次分组结果']].groupby('IVF编号')[['第2次分组结果']].shift(1) #将上行的分组结果下移方便对比
df3_fin['上上行第2次分组结果'] = df3_fin[['IVF编号','第2次分组结果']].groupby('IVF编号')[['第2次分组结果']].shift(2) #将上上行的下移方便对比

In [531]:
#将上单元格迭代多次后，分组结果保持稳定
mycrit2.value_counts()

True     12285
False     8287
dtype: int64

In [540]:
#get_groups
#筛选出每组的标记行（即可作为组内数据合并的标杆行）
df3_key = df3[mycrit2][['IVF编号','检查时间']]
df3_key.columns=['IVF编号','检查组开始时间']
#将每条实验记录附上可能的标记行
df3_mgd = df3.merge(df3_key, on='IVF编号', how='left')
#添加记录实验时间和标记实验组开始时间的时间差
df3_mgd['时间差'] = df3_mgd['检查时间'] - df3_mgd['检查组开始时间']
#根据条件筛选出与每条实验记录最匹配的标记行（即该组实验开始日期）；按照时间差排序，保留时间差最接近0的正值，即可得到最匹配的标记行
df3_mgd = df3_mgd[df3_mgd['时间差'] >= pd.Timedelta(0,unit='d')]\
        .sort_values(['IVF编号','检查时间','时间差']).drop_duplicates(subset=['IVF编号','检查时间'])
#重新设置index
df3_mgd = df3_mgd.reset_index(drop=True)

#### 第三次分组：
观察第二次分组后的结果发现，3行组成的组中，过分合并已经解决。剩余过分合并的均为12312类型的4-5行的组
我们需要找到每组第4行（若存在），将其设为独立组头

In [521]:
#首先将上上上行下移，用来判断每组第四行
df3_fin['上上上行第2次分组结果'] = df3_fin[['IVF编号','第2次分组结果']].groupby('IVF编号')[['第2次分组结果']].shift(3) #将上上行的下移方便对比

In [527]:
#找到有4行以上的组每组的第四行，它们需要设置为新组头，并与上次结果合并
mycrit3 = (((df3_fin['第2次分组结果'] == False) & (df3_fin['上行第2次分组结果'] == False) \
            & (df3_fin['上上行第2次分组结果'] == False) & (df3_fin['上上上行第2次分组结果'] == True)) \
            | (df3_fin['第2次分组结果'] == True))

In [567]:
df3_fin['第3次分组结果'] = mycrit3

In [568]:
#get_groups(df,mycrit)
#筛选出每组的标记行（即可作为组内数据合并的标杆行）
df3_key = df3[mycrit3][['IVF编号','检查时间']]
df3_key.columns=['IVF编号','检查组开始时间']
#将每条实验记录附上可能的标记行
df3_mgd = df3.merge(df3_key, on='IVF编号', how='left')
#添加记录实验时间和标记实验组开始时间的时间差
df3_mgd['时间差'] = df3_mgd['检查时间'] - df3_mgd['检查组开始时间']
#根据条件筛选出与每条实验记录最匹配的标记行（即该组实验开始日期）
#按照时间差排序，保留时间差最接近0的正值，即可得到最匹配的标记行
df3_mgd = df3_mgd[df3_mgd['时间差'] >= pd.Timedelta(0,unit='d')]\
        .sort_values(['IVF编号','检查时间','时间差']).drop_duplicates(subset=['IVF编号','检查时间'])
#重新设置index
df3_mgd = df3_mgd.reset_index(drop=True)

#### 3.检查分组得到的结果：

In [2085]:
#get_sumtab
#按现有分组得到的每人每组数据情况
mytab = df3_mgd[['IVF编号','检查组开始时间','1类实验结果','2类实验结果','3类实验结果']].groupby(['IVF编号','检查组开始时间'])\
        .sum().reset_index()
#加入判断列，记录该组是否有全三项数据
mytab['是否含全3类'] = (mytab['1类实验结果'] >= 1) & (mytab['2类实验结果'] >= 1) & (mytab['3类实验结果'] >= 1)
#加入记录列，记录每组数据行数
mytab['组内数据行数'] = df3_mgd.groupby(['IVF编号','检查组开始时间'])['IVF编号'].count().reset_index(drop=True)
#每人数据总情况
mytab_sum = df3_mgd[['IVF编号','1类实验结果','2类实验结果','3类实验结果']].groupby(['IVF编号']).sum().reset_index()
#判断记录该人是否有全三项数据
mytab_sum['是否含全3类'] = (mytab_sum['1类实验结果'] >= 1) & (mytab_sum['2类实验结果'] >= 1) & (mytab_sum['3类实验结果'] >= 1)
#按现有分组，记录每个人是否至少有一组完整数据
mytab_sum['至少一组含全3类'] = (mytab.groupby('IVF编号')['是否含全3类'].sum() >= 1).reset_index(drop=True)

#### 检查1：
检查某人记录有完整数据，但是分组后没有任何一组是完整数据的组数。检验是否$分组过细$：

In [2086]:
#筛选出：个人全纪录里有完整数据，但是分组后没有一组包含完整数据的组。
myse =mytab_sum[(mytab_sum['是否含全3类'] == True) & (mytab_sum['至少一组含全3类'] == False)]['IVF编号']
#满足以上条件的人数（数量越大说明分组越细）
myse.nunique()

48

In [2087]:
mytab_sum[(mytab_sum['是否含全3类'] == True)]['至少一组含全3类'].value_counts()

True     1958
False      48
Name: 至少一组含全3类, dtype: int64

In [2088]:
mytab_sum[(mytab_sum['是否含全3类'] == False)]['至少一组含全3类'].value_counts()

False    5074
Name: 至少一组含全3类, dtype: int64

#### 检查2：
检查每个分组内的数据行数，用于检查是否$过分合并$，
按理每组数据行数最多为3，某些组行数大于3的原因是：某些数据行与上行结构互补，时间差小，但是该行与上上行重复

In [2089]:
#检查每个分组的所有情况
mytab.describe()

,组内数据行数
count,12319.000000
mean,1.669941
std,0.700357
min,1.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,3.000000


In [2090]:
#检查数据行数为3时，数据点重合情况（数量越大说明过分合并越多）
mytab[((mytab['1类实验结果'] >= 2) | (mytab['2类实验结果'] >= 2) | (mytab['3类实验结果'] >= 2)) & (mytab['组内数据行数']==3)].describe()

,组内数据行数
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [2091]:
#检查数据行数超过3行的情况（数量越大说明过分合并越多）
mytab[mytab['组内数据行数']>3].describe()

,组内数据行数
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [2092]:
#检查所有组中数据点重合情况（数量越大说明过分合并越多）
mytab[((mytab['1类实验结果'] >= 2) | (mytab['2类实验结果'] >= 2) | (mytab['3类实验结果'] >= 2))].describe()

,组内数据行数
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


#### 4.最终数据合并

In [2093]:
mycols = ['姓名', '禁欲天数', '精液量', '精子密度', '精子密度2',\
       'a级 快速前向率', 'b级 慢速前向率', '向前精子率', '非向前精子率', '不动精子率', '形态正常', 'DNA碎片率指数']

In [626]:
#将字符串替换
df3_mgd['禁欲天数'] = df3_mgd['禁欲天数'].replace(['适中','不明','?','？'],np.nan)
df3_mgd['精液量'] = df3_mgd['精液量'].replace('1.1.',1.1)
df3_mgd['精子密度2'] = df3_mgd['精子密度2'].replace({'35-50': 42.5, '45-65': 55, '20-40':30, '50-70':60})
df3_mgd['a级 快速前向率'] = df3_mgd['a级 快速前向率'].replace('N.A.',np.nan)
df3_mgd['b级 慢速前向率'] = df3_mgd['b级 慢速前向率'].replace('N.A.',np.nan)

In [2108]:
#根据IVF、检查组分组，将每组的数据合并
df3_pro = df3_mgd.groupby(['IVF编号','检查组开始时间'])[mycols].mean().reset_index()

In [2109]:
#加入姓名列，加入检查组结束时间列
df3_pro['姓名'] = df3_mgd.groupby(['IVF编号','检查组开始时间'])['姓名'].first().reset_index(drop=True)
df3_pro['检查组结束时间'] = df3_mgd.groupby(['IVF编号','检查组开始时间'])['检查时间'].max().reset_index(drop=True)

In [2203]:
#合并精子密度列和精子密度2列
df3_pro.loc[df3_pro['精子密度'].isna(),'精子密度'] = df3_pro.loc[df3_pro['精子密度'].isna(),'精子密度2']

In [2214]:
#合并向前精子率三列
df3_pro.loc[df3_pro['向前精子率'].isna(),'向前精子率'] = (df3_pro.loc[df3_pro['向前精子率'].isna(), 'a级 快速前向率'] \
                                                        + df3_pro.loc[df3_pro['向前精子率'].isna(), 'b级 慢速前向率'])

In [2217]:
#丢弃多于的那三列，并且重置顺序
df3_pro = df3_pro[['IVF编号', '姓名', '检查组开始时间', '检查组结束时间', \
                   '禁欲天数', '精液量', '精子密度', '向前精子率', '非向前精子率', '不动精子率', '形态正常', 'DNA碎片率指数']]

In [2218]:
#得到最终结果（检查组开始时间：唯一，可作为键）
df3_pro[['IVF编号','检查组开始时间']].describe()

,IVF编号,检查组开始时间
count,12319,12319
unique,7080,1538
top,17-1151,2016-03-04 00:00:00
freq,19,32
first,NaN,2009-02-13 00:00:00
last,NaN,2018-01-02 00:00:00


### 对【胚胎移植记录表】进行处理

In [725]:
#去重
df2 = df2.drop_duplicates()

In [908]:
newcols = list(df2.columns.values)

#### 取卵记录表处理

In [2152]:
#取卵记录
df2_left = df2[newcols[:13]].drop_duplicates(newcols[:1]+newcols[2:13]).reset_index(drop=True)

In [2153]:
#按照IVF号及周期分组，记录每个周期中的行数，以检查是否有重复周期数据。
mytab_left = df2_left.groupby(['IVF编号','周期'])[['实验室创建时间']].count().reset_index()

In [2154]:
#找出取卵组中有多行数据的index，找出明显的重复项
drop_index = list(df2_left[(df2_left['IVF编号'].isin(mytab_left[mytab_left['实验室创建时间']>=2]['IVF编号']) & df2_left['获卵'].isna())].index.values)

In [2155]:
#手动添加需要删除的明显错误的index
drop_index.extend([6871,8308,10428,10429,10477,10513,10735,10767,10972,10986,11042,11096,11604,11644,11882,11948,12052,12219])

In [2156]:
#删除以上选出的行
df2_left = df2_left.drop(drop_index).reset_index(drop=True)

In [2159]:
#接着手动更正明显错误数据
df2_left.loc[1645,'周期'] = '第3周期'
df2_left.loc[1909,'周期'] = '第1周期'
df2_left.loc[3320,'周期'] = '第3周期'
df2_left.loc[10892,'周期'] = '第2周期'
df2_left.loc[11211,'周期'] = '第1周期'
df2_left.loc[11600,'周期'] = '第2周期'
df2_left.loc[12234,'周期'] = '第3周期'
#!陈志英!、!马沁!需要注意，移植表中同样需要更改

In [2163]:
#再次检查：按照IVF号及周期分组，记录每个周期中的行数，以检查是否有重复周期数据。
mytab_left = df2_left.groupby(['IVF编号','周期'])[['实验室创建时间']].count().reset_index()

In [2164]:
#检查发现已经没有重复项目
df2[df2['IVF编号'].isin(mytab_left[mytab_left['实验室创建时间']>=2]['IVF编号'])]

,IVF编号,姓名,实验室创建时间,周期,取卵日期,获卵,MII,授精方式,受精,D2,D3,D5,D6,移植日期,周期.1,移植胚胎个数,移植周期,ART方式,新鲜冷冻创建时间,血β_HCG,TV_B,诊断


In [2165]:
#重新命名列
df2_left = df2_left.rename(columns={'周期':'取卵周期'})

In [2174]:
#找出取卵日期-上一次取卵日期的对应关系
df2_left_timeseries = df2_left[['IVF编号','取卵日期']].drop_duplicates().dropna().reset_index(drop=True)

In [2175]:
#将上一行的移植日期下移，并且将na值填充为过去很久的日期，方便之后的日期比较
df2_left_timeseries['上一次取卵日期'] = df2_left_timeseries.groupby(['IVF编号'])['取卵日期'].shift(1).fillna(pd.Timestamp('1920-01-01'))

In [2180]:
#将上一次取卵日期合并至表格中
df2_left = df2_left.merge(df2_left_timeseries, on=['IVF编号','取卵日期'], how='left')
#重新排序
df2_left = df2_left[['IVF编号', '姓名', '实验室创建时间', '取卵周期', '上一次取卵日期', '取卵日期', '获卵', 'MII', '授精方式', '受精', 'D2', 'D3', 'D5', 'D6']]

In [2668]:
df2_left['获卵'].unique()
#检查发现获卵中存在‘/’的数据，需要替换为nan

array([3, 1, 0, 6, 2, 10, 4, 5, 9, 16, 17, 13, 11, 14, 25, 15, 8, 19, 7,
       22, 12, nan, 26, 44, 21, 36, 20, 23, 18, 30, 29, 33, 24, 34, 27,
       35, 42, 50, 31, 40, 32, 39, 38, 62, '/', 45, 28, 41, 43, 48, 54,
       37, 51, 69, 220, 52, 46], dtype=object)

In [2670]:
#替换‘/’为nan
df2_left['获卵'] = df2_left['获卵'].replace({'/':np.nan})

In [2672]:
#得到了最终结果（'取卵周期'：周期序列号，唯一，可作为键）
df2_left[['IVF编号','取卵周期']].describe()

,IVF编号,取卵周期
count,12302,12302
unique,7916,26
top,15-1217,第1周期
freq,26,7759


#### 移植记录表处理

In [1465]:
#该表包含的列
newcols[:2] + newcols[13:18]

['IVF编号', '姓名', '移植日期', '周期.1', '移植胚胎个数', '移植周期', 'ART方式']

In [1466]:
#将这些列选出作为移植记录表
df2_mid = df2[newcols[:2] + newcols[13:18]]

In [1467]:
#检查发现没有周期.1数据的整行都没有数据，找出这些行
drop_index_mid = df2_mid[df2_mid['周期.1'].isna()].index

In [1468]:
#删除以上空行并且去重
df2_mid = df2_mid.drop(drop_index_mid).drop_duplicates().reset_index(drop=True)

In [1117]:
#检查每个人的每个周期数据是否还有无法轻易发现的重复项
mytab_mid = df2_mid.groupby(['IVF编号','周期.1','移植日期'])[['周期.1']].count()
mytab_mid.columns = ['count']

In [1118]:
#观察重复项，手动删除
df2_mid[df2_mid['IVF编号'].isin(mytab_mid[mytab_mid['count']>=2].reset_index()['IVF编号'])]

,IVF编号,姓名,移植日期,周期.1,移植胚胎个数,移植周期,ART方式
1159,15-0499,倪娟娟,2015-11-23,第2周期,1.0,D6,FET
1160,15-0499,倪娟娟,2016-03-03,第2周期,1.0,D6,FET
1161,15-0499,倪娟娟,2016-03-03,第2周期,1.0,D3,FET
1162,15-0499,倪娟娟,2016-05-06,第2周期,1.0,D6*1,FET
2606,15-2197,张媛,2016-03-28,第1周期,2.0,D6,FET
2607,15-2197,张媛,2016-03-28,第1周期,NaN,D6,FET
2608,15-2197,张媛,2016-07-29,第2周期,1.0,D3,FET


In [1469]:
#手动删除
df2_mid = df2_mid.drop([1161,2607]).reset_index(drop=True)

In [1470]:
#重命名周期列
df2_mid = df2_mid.rename(columns={'周期.1':'取卵周期'})

In [1998]:
#取卵表中残留的问题：15-1436 陈， 18-0441 马
df2_mid.loc[((df2_mid['IVF编号'] == '15-1436') & (df2_mid['取卵周期'] == '第2周期')),'取卵周期'] = '第3周期'
df2_mid.loc[((df2_mid['IVF编号'] == '18-0441') & (df2_mid['取卵周期'] == '第1周期')),'取卵周期'] = '第2周期'

#### 诊断记录表处理

In [1175]:
#选出需要的列
df2_right = df2[newcols[:2] + newcols[18:]]

In [1176]:
#经观察发现无新鲜冷冻创建时间的其他项也均无数据，故删去
drop_index_right = df2_right[df2_right['新鲜冷冻创建时间'].isna()].index

In [1177]:
#删去以上的数据条
df2_right = df2_right.drop(drop_index_right).sort_values(['IVF编号','新鲜冷冻创建时间']).reset_index(drop=True)

In [1178]:
df2_right.describe()
#检验发现创建时间无重复项，处理完成；保留了仅有创建时间但无其他数据的条目，可用来作为没有成功移植的潜在判断条件

,IVF编号,姓名,新鲜冷冻创建时间,血β_HCG,TV_B,诊断
count,7105,6327,7105,6686.0,3358,3519
unique,5969,4824,7105,3190.0,3260,1841
top,15-0467,袁媛,2017-12-22 13:48:55.900000,0.1,",","早孕,早孕"
freq,6,10,1,1119.0,54,339
first,NaN,NaN,2014-12-24 16:42:03.323000,NaN,NaN,NaN
last,NaN,NaN,2018-09-17 15:18:44.293000,NaN,NaN,NaN


#### 合并移植记录和诊断记录表格

首先找移植和诊断的对照关系

In [1500]:
#单独调出移植信息
df2_imp = df2_mid[['IVF编号','姓名','移植日期']].drop_duplicates().dropna().reset_index(drop=True)

In [1501]:
#将下一行的移植日期上移，并且将na值填充为未来的日期，方便之后与诊断日期比较
df2_imp['下一次移植日期'] = df2_imp.groupby(['IVF编号'])['移植日期'].shift(-1).fillna(pd.Timestamp('2020-01-01'))

In [1502]:
#接下来要找到移植和诊断的一一对应关系：
#步骤1：先将两者做以移植信息为基础的左合并
df2_key = df2_imp.merge(df2_right[['IVF编号','新鲜冷冻创建时间']], on='IVF编号', how='left')

In [1503]:
#步骤2：筛选出其中满足移植日期和下一次移植日期间隔中间的诊断时间
df2_key = df2_key[(df2_key['新鲜冷冻创建时间'] > df2_key['移植日期']) & (df2_key['新鲜冷冻创建时间'] < df2_key['下一次移植日期'])]

In [1941]:
#检查其中一个移植日期对应了多个诊断的条目
mytab_new = df2_key.groupby(['IVF编号','移植日期'])[['新鲜冷冻创建时间']].count().reset_index()
mytab_new = mytab_new.rename(columns={'新鲜冷冻创建时间':'count'})

In [1953]:
#对于一个移植日期对应多个诊断的，只保留最近的那个诊断
df2_key = df2_key.drop_duplicates(subset=['IVF编号','移植日期']).reset_index(drop=True)

合并移植和诊断信息

In [1955]:
#添加下一次移植日期
df2_merge = df2_mid.merge(df2_imp[['IVF编号','移植日期','下一次移植日期']], on=['IVF编号','移植日期'], how='left')
#步骤1：首先把新鲜冷冻创建时间加入df2_mid中作为匹配的键
df2_merge = df2_merge.merge(df2_key[['IVF编号','移植日期','新鲜冷冻创建时间']], on=['IVF编号','移植日期'], how='left')
#步骤2：以诊断时间为键外合并，保证未映射的诊断信息也被添加。最后重新排序。
df2_merge = df2_merge.merge(df2_right[['IVF编号','新鲜冷冻创建时间','血β_HCG','TV_B','诊断']], on=['IVF编号','新鲜冷冻创建时间'],how='outer')\
        .sort_values(by=['IVF编号','移植日期','新鲜冷冻创建时间']).reset_index(drop=True)

In [1956]:
#首先将下一次移植日期中的2020替换回原来的na，方便之后操作比较
df2_merge['下一次移植日期'] = df2_merge['下一次移植日期'].replace(pd.Timestamp('2020-01-01'),np.nan)

In [1884]:
df2_merge['IVF编号'].describe()
#得到了最终移植和诊断信息合并结果df2_merge

count        8915
unique       6685
top       14-1409
freq            9
Name: IVF编号, dtype: object

#### 处理移植诊断记录，推断受孕结果

In [1965]:
#判断移植成功条件（排除生化妊娠）。记录移植成功列，加入表中
df2_merge['移植成功'] = ((df2_merge['TV_B'].str.contains('孕|胚|胎|囊|芽|CRL|心|流产') == True)\
                         | (df2_merge['诊断'].str.contains('孕|胎|流产|异位妊娠') == True))\
                         & ((df2_merge['诊断'].str.contains('生化妊娠') == False)\
                         | (df2_merge['诊断'].isna()))

In [1966]:
#判断是否双胎，加入表中。
df2_merge['双胎'] = ((df2_merge['TV_B'].str.contains('孕|胚|胎|囊|芽|CRL|心|流产') == True)\
                         | (df2_merge['诊断'].str.contains('孕|胎|流产|异位妊娠') == True))\
                         & ((df2_merge['诊断'].str.contains('生化妊娠') == False)\
                         | (df2_merge['诊断'].isna()))\
                         & ((df2_merge['移植胚胎个数']>=2) | (df2_merge['移植胚胎个数'].isna()))\
                         & ((df2_merge['TV_B'].str.contains('双')==True) | (df2_merge['诊断'].str.contains('双')==True))

In [1967]:
#加入移植成功个数
df2_merge['移植成功个数'] = df2_merge['移植成功'].map({True:1,False:0}) + df2_merge['双胎'].map({True:1,False:0})

In [1928]:
###没有诊断记录的那些条目，是当做没有受孕处理，还是舍弃这些数据？###
#暂时当做没有受孕处理

In [1970]:
mytab_mgd = df2_merge.groupby(['IVF编号','移植日期'])[['移植日期']].count()
mytab_mgd.columns = ['总行数']
mytab_mgd = mytab_mgd.reset_index()

In [2072]:
#检查同一移植日期有多行记录的
df2_merge[df2_merge['IVF编号'].isin(mytab_mgd[mytab_mgd['总行数']>=2]['IVF编号'])]
#经检查发现诊断记录和判断是否受孕没有错误

,IVF编号,姓名,移植日期,取卵周期,移植胚胎个数,移植周期,ART方式,下一次移植日期,新鲜冷冻创建时间,血β_HCG,TV_B,诊断,移植成功,双胎,移植成功个数
843,15-0011,岳金霞,2015-08-03,第1周期,1.0,D3,FET,NaT,NaT,NaN,NaN,NaN,False,False,0
844,15-0011,岳金霞,2015-08-03,第2周期,1.0,D3,FET,NaT,NaT,NaN,NaN,NaN,False,False,0
1238,15-0414,马晓梅,2016-05-17,第2周期,1.0,D6,FET,NaT,2016-05-31 15:41:17.927,0.2,NaN,NaN,False,False,0
1239,15-0414,马晓梅,2016-05-17,第3周期,1.0,D6,FET,NaT,2016-05-31 15:41:17.927,0.2,NaN,NaN,False,False,0
1240,15-0414,NaN,NaT,NaN,NaN,NaN,NaN,NaT,2016-06-08 09:25:33.647,0.2,NaN,NaN,False,False,0
2000,15-1135,毛玉怡,2016-01-26,第1周期,NaN,D3,FET,NaT,2016-09-23 10:22:32.050,TβHCG2:0.2 U/L,NaN,NaN,False,False,0
2001,15-1135,毛玉怡,2016-01-26,第2周期,1.0,D3,FET,NaT,2016-09-23 10:22:32.050,TβHCG2:0.2 U/L,NaN,NaN,False,False,0
2002,15-1135,NaN,NaT,NaN,NaN,NaN,NaN,NaT,2015-11-04 12:57:17.640,0.1,NaN,NaN,False,False,0
2171,15-1280,孙燕,2017-04-25,第2周期,1.0,D3,FET,NaT,NaT,NaN,NaN,NaN,False,False,0
2172,15-1280,孙燕,2017-04-25,第4周期,1.0,D3,FET,NaT,NaT,NaN,NaN,NaN,False,False,0


In [2247]:
#得到了移植和诊断受孕的处理后结果df2_merge，且没有删除缺数据的条目，使用时仅需要添加筛选条件即可
df2_merge[['IVF编号','移植日期','取卵周期']].describe()

,IVF编号,移植日期,取卵周期
count,8913,7323,7324
unique,6685,1263,18
top,14-1409,2017-02-28 00:00:00,第1周期
freq,9,20,5442
first,NaN,2014-09-02 00:00:00,NaN
last,NaN,2018-09-21 00:00:00,NaN


### 合并表格

In [2080]:
#表1：男女基本信息
df1_pro['IVF编号'].describe()

count        8770
unique       8770
top       18-0960
freq            1
Name: IVF编号, dtype: object

In [2113]:
#表2：男性精液检查记录
df3_pro['IVF编号'].describe()

count       12319
unique       7080
top       17-1151
freq           19
Name: IVF编号, dtype: object

In [2182]:
#表2：取卵记录
df2_left['IVF编号'].describe()

count       12302
unique       7916
top       15-1217
freq           26
Name: IVF编号, dtype: object

In [2074]:
#表3：移植和诊断记录
df2_merge['IVF编号'].describe()

count        8913
unique       6685
top       14-1409
freq            9
Name: IVF编号, dtype: object

#### 合并取卵、精液、基本信息记录

In [2673]:
#左合并取卵记录和精液记录
df_fert = df2_left.merge(df3_pro, on='IVF编号', how='left')

In [2674]:
#筛出在两次取卵之间的精液记录，并且保留最接近的那一次
df_fert = df_fert[(df_fert['检查组开始时间'] > df_fert['上一次取卵日期']) & (df_fert['检查组结束时间'] <= df_fert['取卵日期'])]\
            .drop_duplicates(subset=['IVF编号','取卵周期'], keep='last')

In [2675]:
#合并入男女方基本信息
df_fert = df_fert.merge(df1_pro, on='IVF编号', how='left')

In [2676]:
#得到了最终取卵授精结果表
df_fert[['IVF编号','取卵周期','检查组开始时间']].describe()

,IVF编号,取卵周期,检查组开始时间
count,6833,6833,6833
unique,6376,12,1037
top,16-1421,第1周期,2017-02-17 00:00:00
freq,4,6194,23
first,NaN,NaN,2009-12-14 00:00:00
last,NaN,NaN,2018-01-02 00:00:00


In [2684]:
#筛选后的结果表格
df_fert_fil = df_fert[(df_fert['取卵周期'].str.contains('第1周期|第2周期|第3周期')) \
        & ((df_fert['取卵日期'] - df_fert['女方生日']) <= pd.Timedelta(35*365, 'd')) \
        & (df_fert['诊断'].str.contains('染色体异常|PCOS|排卵障碍|子宫内膜异位症') == False) \
        & ((df_fert['授精方式']=='ICSI,') | (df_fert['授精方式']=='IVF,'))\
        & (df_fert['形态正常'].notna())\
        & (df_fert['孕育史'].str.contains('[0-9]-[0-9]-[3-9]-[0-9]|[0-9]-[0-9]-1[0-9]-[0-9]') == False)\
        & (df_fert['获卵'] <= 30)]

In [2689]:
df_fert_fil[['IVF编号','取卵周期','检查组开始时间']].describe()

,IVF编号,取卵周期,检查组开始时间
count,2502,2502,2502
unique,2410,3,751
top,16-2241,第1周期,2017-06-27 00:00:00
freq,3,2348,14
first,NaN,NaN,2014-06-10 00:00:00
last,NaN,NaN,2017-12-22 00:00:00


#### 合并移植诊断表(df2_merge)和取卵授精表(df_fert)

In [2685]:
#内合并移植诊断表和取卵授精表
df_preg = df2_merge.merge(df_fert, on=['IVF编号','取卵周期'], how='inner')

In [2686]:
#得到了最终受孕结果表
df_preg[['IVF编号','取卵周期']].describe()

,IVF编号,取卵周期
count,4714,4714
unique,4235,8
top,15-2022,第1周期
freq,4,4458


In [2708]:
#筛选后的结果（新增对移植和诊断信息的筛选）
df_preg_fil = df_preg[(df_preg['取卵周期'].str.contains('第1周期|第2周期|第3周期')) \
        & ((df_preg['取卵日期'] - df_preg['女方生日']) <= pd.Timedelta(35*365, 'd')) \
        & (df_preg['诊断_y'].str.contains('染色体异常|PCOS|排卵障碍|子宫内膜异位症') == False) \
        & ((df_preg['授精方式']=='ICSI,') | (df_preg['授精方式']=='IVF,'))\
        & (df_preg['形态正常'].notna())\
        & (df_preg['孕育史'].str.contains('[0-9]-[0-9]-[3-9]-[0-9]|[0-9]-[0-9]-1[0-9]-[0-9]') == False)\
        & (df_preg['获卵'] <= 30)\
        & (df_preg['移植胚胎个数'].notna())\
        & (df_preg['新鲜冷冻创建时间'].notna() | df_preg['下一次移植日期'].notna())]

In [2709]:
df_preg_fil['IVF编号'].describe()

count        1615
unique       1476
top       16-1099
freq            3
Name: IVF编号, dtype: object

### 处理产科数据

In [2430]:
#匹配受孕结果和产科数据表

In [2529]:
#更改列名称 方便之后匹配
df4_pro = df4.rename(columns={'姓名':'女方姓名','出生年月':'女方生日'})

In [2542]:
#以女方姓名和女方生日为键，匹配合并
df_labor = df4_pro.merge(df_preg, on=['女方姓名','女方生日'], how='inner')

In [2543]:
#添加出生-移植时间差，用作比较
df_labor['出生时间-移植日期'] = df_labor['出生时间1'] - df_labor['移植日期']

In [2544]:
#换算孕周为天数
df_labor['孕周'] = df_labor['孕周'].apply(lambda x: int(x) * 7 + x % 1 *10)
df_labor['孕周'] = df_labor['孕周'].apply(lambda x: pd.Timedelta(x, unit='d'))

In [2710]:
#筛选后的结果
df_labor_fil = df_labor[(df_labor['取卵周期'].str.contains('第1周期|第2周期|第3周期')) \
        & ((df_labor['取卵日期'] - df_labor['女方生日']) <= pd.Timedelta(35*365, 'd')) \
        & (df_labor['诊断_y'].str.contains('染色体异常|PCOS|排卵障碍|子宫内膜异位症') == False) \
        & ((df_labor['授精方式']=='ICSI,') | (df_labor['授精方式']=='IVF,'))
        & (df_labor['形态正常'].notna())\
        & (df_labor['孕育史'].str.contains('[0-9]-[0-9]-[3-9]-[0-9]|[0-9]-[0-9]-1[0-9]-[0-9]') == False)\
        & (df_labor['获卵'] <= 30)\
        & (df_labor['孕周'] > df_labor['出生时间-移植日期'])]

In [2716]:
df_labor_fil.shape

(299, 121)

In [2720]:
#save to csv
df_fert_fil.to_csv('fert_fil.csv', index=False, encoding='ANSI')
df_preg_fil.to_csv('preg_fil.csv', index=False, encoding='ANSI')
df_labor_fil.to_csv('labor_fil.csv', index=False, encoding='ANSI')